# Debugging autoreload

In [1]:
%load_ext autoreload
%autoreload 2

# Load packages

In [15]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from sklearn.model_selection import BaseCrossValidator, ParameterGrid, ParameterSampler
import torch
import pickle
import shutil
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from sklearn.model_selection import train_test_split
import numpy as np
from pytorch_tabular.utils import make_mixed_dataset, print_metrics
from pytorch_tabular import available_models
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GANDALFConfig, TabNetModelConfig, FTTransformerConfig, DANetConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.tabular_model_tuner import TabularModelTuner
from torchmetrics.functional.regression import mean_absolute_error, pearson_corrcoef
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.utils.hash import dict_hash
from src.pt.hyper_opt import train_hyper_opt
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
import optuna
from plottable import ColumnDefinition, Table
from plottable.plots import bar
from plottable.cmap import normed_cmap, centered_cmap
import matplotlib.cm


def make_rgb_transparent(rgb, bg_rgb, alpha):
    return [alpha * c1 + (1 - alpha) * c2 for (c1, c2) in zip(rgb, bg_rgb)]

# Comparing multiple features sets

In [5]:
path = f"D:/YandexDisk/Work/bbd/mriya"

feats_sets_models = {
    'Sphygmocardiography': '876',
    'Echocardiography': '162',
    'ECG': '355',
    'BP': '1005',
    'CBC': '160',
    'Anthropometry': '62'
}

colors = distinctipy.get_colors(len(feats_sets_models), [mcolors.hex2color(mcolors.CSS4_COLORS['black']), mcolors.hex2color(mcolors.CSS4_COLORS['white'])], rng=1337, pastel_factor=0.15)
colors_feats_sets = {x: colors[x_id] for x_id, x in enumerate(feats_sets_models.keys())}

## Age histograms

In [ ]:
n_rows = 2
n_cols = 3
fig_width = 15
fig_height = 9
hist_bins = np.linspace(5, 115, 23)

sns.set_theme(style='ticks')
fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), gridspec_kw={}, sharey=True, sharex=True)
for fs_id, (fs, model_id) in enumerate(feats_sets_models.items()):
    row_id, col_id = divmod(fs_id, n_cols)
    
    df_res = pd.read_excel(f"{path}/{fs}/models/trn_val_tst_selection/{model_id}/df.xlsx", index_col=0)
    
    histplot = sns.histplot(
        data=ddf_resf,
        bins=hist_bins,
        edgecolor='k',
        linewidth=1,
        x="Age",
        color=colors_feats_sets[fs],
        ax=axs[row_id, col_id]
    )
    axs[row_id, col_id].set(xlim=(0, 120))
    axs[row_id, col_id].set_title(f"{fs} ({df_res.shape[0]})")
fig.tight_layout()    
fig.savefig(f"{path}/hist_age_for_feats.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path}/hist_age_for_feats.pdf", bbox_inches='tight')
plt.close(fig)

## Scatters and KDEs with metrics

In [ ]:
n_rows = 2 * 3
n_cols = 3
fig_width = 20
fig_height = 12

sns.set_theme(style='ticks')
fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), height_ratios=[0.2, 0.8, 0.2]*2, gridspec_kw={'wspace':0.35, 'hspace': 0.05}, sharey=False, sharex=False)

df_metrics_global = pd.DataFrame(index=list(feats_sets_models.keys()), columns=['MAE', 'Rho', 'Bias'])

for fs_id, (fs, model_id) in enumerate(feats_sets_models.items()):
    row_id, col_id = divmod(fs_id, n_cols)
    row_id_table = row_id * 3
    row_id_scatter = row_id * 3 + 1
    row_id_empty = row_id * 3 + 2
    
    fs_color = colors_feats_sets[fs]
    
    df_results = pd.read_excel(f"{path}/{fs}/models/trn_val_tst_selection/{model_id}/df.xlsx", index_col=0)
    df_metrics = pd.read_excel(f"{path}/{fs}/models/trn_val_tst_selection/{model_id}/metrics.xlsx", index_col=0)
    
    df_metrics_global.at[fs, 'MAE'] = df_metrics.at['Test', 'mean_absolute_error_unbiased']
    df_metrics_global.at[fs, 'Rho'] = df_metrics.at['Test', 'pearson_corrcoef_unbiased']
    df_metrics_global.at[fs, 'Bias'] = df_metrics.at['Test', 'bias_unbiased']
    
    xy_min = df_results[['Age', 'Prediction Unbiased']].min().min()
    xy_max = df_results[['Age', 'Prediction Unbiased']].max().max()
    xy_ptp = xy_max - xy_min

    df_table = pd.DataFrame(index=['MAE', fr"Pearson $\mathbf{{\rho}}$", "Bias"], columns=['Train', 'Validation', 'Test'])
    for part in ['Train', 'Validation', 'Test']:
        df_table.at['MAE', part] = f"{df_metrics.at[part, 'mean_absolute_error_unbiased']:0.3f}"
        df_table.at[fr"Pearson $\mathbf{{\rho}}$", part] = f"{df_metrics.at[part, 'pearson_corrcoef_unbiased']:0.3f}"
        df_table.at["Bias", part] = f"{df_metrics.at[part, 'bias_unbiased']:0.3f}"
    
    col_defs = [
        ColumnDefinition(
            name="index",
            title='',
            textprops={"ha": "center", "weight": "bold"},
            width=2.5,
            # border="both",
            group=fr"$\mathbf{{{fs}}}$",
        ),
        ColumnDefinition(
            name="Train",
            textprops={"ha": "left"},
            width=1.5,
            border="left",
            group=fr"$\mathbf{{{fs}}}$",
        ),
        ColumnDefinition(
            name="Validation",
            textprops={"ha": "left"},
            width=1.5,
            group=fr"$\mathbf{{{fs}}}$",
        ),
        ColumnDefinition(
            name="Test",
            textprops={"ha": "left"},
            width=1.5,
            group=fr"$\mathbf{{{fs}}}$",
        )
    ]

    table = Table(
        df_table,
        column_definitions=col_defs,
        row_dividers=True,
        footer_divider=False,
        ax=axs[row_id_table, col_id],
        textprops={"fontsize": 8},
        row_divider_kw={"linewidth": 1, "linestyle": (0, (1, 1))},
        col_label_divider_kw={"linewidth": 1, "linestyle": "-"},
        column_border_kw={"linewidth": 1, "linestyle": "-"},
    ).autoset_fontcolors(colnames=['Train', 'Validation', 'Test'])

    kdeplot = sns.kdeplot(
        data=df_res.loc[df_res['Group'] != 'Test', :],
        x='Age',
        y='Prediction Unbiased',
        fill=True,
        cbar=False,
        color=fs_color,
        cut=0,
        legend=False,
        ax=axs[row_id_scatter, col_id]
    )
    scatter = sns.scatterplot(
        data=df_res.loc[df_res['Group'] == 'Test', :],
        x='Age',
        y="Prediction Unbiased",
        linewidth=0.5,
        alpha=0.8,
        edgecolor="k",
        s=35,
        color=fs_color,
        ax=axs[row_id_scatter, col_id],
    )
    axs[row_id_scatter, col_id].axline((0, 0), slope=1, color="black", linestyle=":")
    axs[row_id_scatter, col_id].set_xlim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
    axs[row_id_scatter, col_id].set_ylim(xy_min - 0.1 * xy_ptp, xy_max + 0.1 * xy_ptp)
    
    axs[row_id_empty, col_id].axis('off')

fig.tight_layout()    
fig.savefig(f"{path}/models_for_feats.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path}/models_for_feats.pdf", bbox_inches='tight')
plt.close(fig)

col_defs = [
    ColumnDefinition(
        name="index",
        title="Features Set",
        textprops={"ha": "right", "weight": "bold"},
        width=2.25,
    ),
    ColumnDefinition(
        name="MAE",
        title="Test\nMAE",
        textprops={"ha": "center"},
        formatter="{:.3f}",
        cmap=normed_cmap(df_metrics_global["MAE"].dropna(), cmap=matplotlib.cm.Reds, num_stds=2.5),
        width=1.0,
    ),
    ColumnDefinition(
        name="Rho",
        title="Test\n" + r"Pearson $\rho$",
        textprops={"ha": "center"},
        formatter="{:.3f}",
        cmap=normed_cmap(df_metrics_global["Rho"].dropna(), cmap=matplotlib.cm.Greens, num_stds=2.5),
        width=1.0,
        border="left"
    ),
    ColumnDefinition(
        name="Bias",
        title="Test\nBias",
        textprops={"ha": "center"},
        formatter="{:.3f}",
        cmap=centered_cmap(df_metrics_global["Bias"].dropna(), cmap=matplotlib.cm.seismic, num_stds=2.5),
        width=1.0,
        border="left"
    ),
]
# df_metrics_global.insert(0, 'Feature Set', df_metrics_global.index)

fig, ax = plt.subplots()
table = Table(
    df_metrics_global,
    column_definitions=col_defs,
    row_dividers=True,
    footer_divider=False,
    odd_row_color="#ffffff",
    even_row_color="#f0f0f0",
    ax=ax,
    # textprops={"fontsize": 10},
    row_divider_kw={"linewidth": 1, "linestyle": (0, (1, 1))},
    col_label_divider_kw={"linewidth": 1, "linestyle": "-"},
    column_border_kw={"linewidth": 1, "linestyle": "-"},
).autoset_fontcolors(colnames=df_metrics_global.columns.to_list())
fig.savefig(f"{path}/test_metrics_for_feats.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path}/test_metrics_for_feats.pdf", bbox_inches='tight')
plt.close(fig)

## Features importance

In [21]:
from sqlalchemy import column


n_rows = 1
n_cols = 6
fig_width = 30
fig_height = 12

sns.set_theme(style='ticks')
fig, axs = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height), sharey=False, sharex=False)

for fs_id, (fs, model_id) in enumerate(feats_sets_models.items()):
    row_id, col_id = divmod(fs_id, n_cols)
    
    fs_color = colors_feats_sets[fs]
    
    df_results = pd.read_excel(f"{path}/{fs}/models/trn_val_tst_selection/{model_id}/df.xlsx", index_col=0)
    df_metrics = pd.read_excel(f"{path}/{fs}/models/trn_val_tst_selection/{model_id}/metrics.xlsx", index_col=0)
    df_explain = pd.read_excel(f"{path}/{fs}/models/trn_val_tst_selection/{model_id}/explanation.xlsx", index_col=0)
    
    df_fi = pd.DataFrame(index=df_explain.columns.to_list(), columns=['abs(|SHAP|)'])
    for f in df_explain.columns:
        df_fi.at[f, 'abs(|SHAP|)'] = df_explain[f].abs().mean()
    df_fi.sort_values(['abs(|SHAP|)'], ascending=[False], inplace=True)
    
    df_fi = df_fi.head(30)
    df_fi['Features'] = df_fi.index.values
    barplot = sns.barplot(
        data=df_fi,
        x='abs(|SHAP|)',
        y='Features',
        color=fs_color,
        edgecolor='black',
        dodge=False,
        ax=axs[col_id]
    )
    for container in barplot.containers:
        barplot.bar_label(container, label_type='edge', fmt='%0.2f', fontsize=12, padding=2.0)
    axs[col_id].set_title(fs, fontsize='large')
    axs[col_id].set_ylabel('')

fig.tight_layout()    
fig.savefig(f"{path}/barplot_importance_for_feats.png", bbox_inches='tight', dpi=200)
fig.savefig(f"{path}/barplot_importance_for_feats.pdf", bbox_inches='tight')
plt.close(fig)